The MNIST dataset consists of a training set of 60000 examples and test set of 10000 examples. All digits have been size-normalized and centered in a fixed image of 28X28 size. In original dataset, each pixel in the image is represented by an integer between 0 and 255, where 0 is black, 255 is white and anything between represents different shade of gray.
We will split the training set of 60000 examples into two sets. First set of 50000 randomly sampled examples will be used for training the neural network. The remainder 10000 examples will be used as a validation set to estimate the hyper-parameters of the network (regularization constant λ, number of hidden units).

The pixel values are gray scale between 0 and 255. It is almost always a good idea to perform some scaling of input values when using neural network models. Because the scale is well known and well behaved, we can very quickly normalize the pixel values to the range 0 and 1 by dividing each value by the maximum of 255.

Since we have 10 possible digits, there are 10 units in the output layer denoted by k and n training samples.

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from math import sqrt
import time
from datetime import timedelta
from keras.utils import np_utils

C:\Users\mmishra\AppData\Local\conda\conda\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def initializeWeights(n_in, n_out):
    """
    This function returns the random weights for Neural Network given the
    number of node in the input layer and output layer

    Input:
    n_in: number of nodes of the input layer
    n_out: number of nodes of the output layer

    Output:
    W: matrix of random initial weights with size (n_out x (n_in + 1))
    """

    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W

In [3]:
def sigmoid(z):
    """
    Notice that z can be a scalar, a vector or a matrix return the sigmoid of input z
    """
    return  1.0/(1.0+np.exp(-z))

In [62]:
def preprocess():
    """ 
    Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the
       training set
     test_data: matrix of training set. Each row of test_data contains
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set

     Some suggestions for preprocessing step:
     - feature selection
     """

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary
    # Pick a reasonable size for validation data

    # ------------Initialize preprocess arrays----------------------#
    train_preprocess = np.zeros(shape=(50000, 784))
    validation_preprocess = np.zeros(shape=(10000, 784))
    test_preprocess = np.zeros(shape=(10000, 784))
    train_label_preprocess = np.zeros(shape=(50000,))
    validation_label_preprocess = np.zeros(shape=(10000,))
    test_label_preprocess = np.zeros(shape=(10000,))
    # ------------Initialize flag variables----------------------#
    train_len = 0
    validation_len = 0
    test_len = 0
    train_label_len = 0
    validation_label_len = 0
    # ------------Start to split the data set into 6 arrays-----------#
    for key in mat:
        # -----------when the set is training set--------------------#
        if "train" in key:
            label = key[-1]  # record the corresponding label
            tup = mat.get(key)
            sap = range(tup.shape[0])
            tup_perm = np.random.permutation(sap)
            tup_len = len(tup)  # get the length of current training set
            tag_len = tup_len - 1000  # defines the number of examples which will be added into the training set

            # ---------------------adding data to training set-------------------------#
            train_preprocess[train_len:train_len + tag_len] = tup[tup_perm[1000:], :]
            train_len += tag_len

            train_label_preprocess[train_label_len:train_label_len + tag_len] = label
            train_label_len += tag_len

            # ---------------------adding data to validation set-------------------------#
            validation_preprocess[validation_len:validation_len + 1000] = tup[tup_perm[0:1000], :]
            validation_len += 1000

            validation_label_preprocess[validation_label_len:validation_label_len + 1000] = label
            validation_label_len += 1000

            # ---------------------adding data to test set-------------------------#
        elif "test" in key:
            label = key[-1]
            tup = mat.get(key)
            sap = range(tup.shape[0])
            tup_perm = np.random.permutation(sap)
            tup_len = len(tup)
            test_label_preprocess[test_len:test_len + tup_len] = label
            test_preprocess[test_len:test_len + tup_len] = tup[tup_perm]
            test_len += tup_len
            # ---------------------Shuffle,double and normalize-------------------------#
    train_size = range(train_preprocess.shape[0])
    train_perm = np.random.permutation(train_size)
    train_data = train_preprocess[train_perm]
    train_data = np.double(train_data)
    train_data = train_data / 255.0
    train_label = train_label_preprocess[train_perm]

    validation_size = range(validation_preprocess.shape[0])
    vali_perm = np.random.permutation(validation_size)
    validation_data = validation_preprocess[vali_perm]
    validation_data = np.double(validation_data)
    validation_data = validation_data / 255.0
    validation_label = validation_label_preprocess[vali_perm]

    test_size = range(test_preprocess.shape[0])
    test_perm = np.random.permutation(test_size)
    test_data = test_preprocess[test_perm]
    test_data = np.double(test_data)
    test_data = test_data / 255.0
    test_label = test_label_preprocess[test_perm]

    # Feature selection
    selectedFeat = []
    rejectedFeat = []
    same = []
    for i in range(train_preprocess.shape[1]):
        topVal = train_data[0,i]
        for j in range(train_preprocess.shape[0]):
            same.append(train_data[j,i] == topVal)
        if (False in same):
            selectedFeat.append(i)
        else:
            rejectedFeat.append(i)
        same = []

    train_data = np.delete(train_data, rejectedFeat, axis = 1)
    validation_data = np.delete(validation_data, rejectedFeat, axis = 1)  
    test_data = np.delete(test_data, rejectedFeat, axis = 1)
    
    print('preprocess done')

    return train_data, train_label, validation_data, validation_label, test_data, test_label

In [69]:
def nnObjFunction(params, *args):
    """
    This function computes the value of objective function (negative log
    likelihood error function with regularization) given the parameters
    of Neural Networks, thetraining data, their corresponding training
    labels and lambda - regularization hyper-parameter.

    Input:
    params: vector of weights of 2 matrices w1 (weights of connections from
            input layer to hidden layer) and w2 (weights of connections from
            hidden layer to output layer) where all of the weights are contained
            in a single vector.
    n_input: number of node in input layer (not include the bias node)
    n_hidden: number of node in hidden layer (not include the bias node)
    n_class: number of node in output layer (number of classes in
             classification problem
    training_data: matrix of training data. Each row of this matrix
                   represents the feature vector of a particular image
    training_label: the vector of truth label of training images. Each entry
                    in the vector represents the truth label of its corresponding image.
    lambda: regularization hyper-parameter. This value is used for fixing the
            overfitting problem.

    Output:
    obj_val: a scalar value representing value of error function
    obj_grad: a SINGLE vector of gradient value of error function

    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    reshape 'params' vector into 2 matrices of weight w1 and w2
    w1: matrix of weights of connections from input layer to hidden layers.
        w1(i, j) represents the weight of connection from unit j in input
        layer to unit i in hidden layer.
    w2: matrix of weights of connections from hidden layer to output layers.
        w2(i, j) represents the weight of connection from unit j in hidden
        layer to unit i in output layer.
    
    """

    n_input, n_hidden, n_class, training_data, training_label, lambdaval = args

    w1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
    w2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))

    obj_val = 0

    # feedforward
    n = training_data.shape[0]
    X = np.concatenate((training_data, np.tile([1], (training_data.shape[0], 1))), 1)
    z = sigmoid(np.dot(X, w1.T))
    z = np.concatenate((z, np.tile([1], (z.shape[0], 1))), 1)
    o = sigmoid(np.dot(z,w2.T))
    
    
    # 1-of-K coding scheme of input data
    y = np_utils.to_categorical(training_label)
    
    obj_val = obj_val + np.sum(y * np.log(o) + (1.0-y) * np.log(1.0-o))
    obj_val = -1*obj_val/n
    
    # with regularization hyper-parameter
    obj_val = obj_val + ((lambdaval/(2*n))*(np.sum(np.square(w1))+np.sum(np.square(w2))))
    

    delta = o-y
    
    grad_w2 = np.dot(delta.T,z)
    grad_w1 = np.dot(((1-z)*z*(np.dot(delta,w2))).T,X)  
    
    # Remove zero row
    grad_w1 = np.delete(grad_w1, n_hidden,0)

    # with regularization hyper-parameter
    grad_w2 = (grad_w2 + (lambdaval*w2))*(1/n)

    grad_w1 = (grad_w1 + (lambdaval*w1))*(1/n)

    obj_grad = np.array([])

    obj_grad = np.concatenate((grad_w1.flatten(), grad_w2.flatten()),0)

    return (obj_val, obj_grad)

In [42]:
def nnPredict(w1, w2, data):
    """
    Predicts the label of data given the parameter w1, w2 of Neural Network.
    """ 
    n = data.shape[0]
    X = np.hstack((data,np.ones((n,1))))
    a = np.dot(w1,X.T)
    z = sigmoid(a)
    z = np.vstack((z,np.ones((1,n))))
    b = np.dot(w2,z)
    o = np.transpose(sigmoid(b))
    # take value corresponding with max probability
    labels = np.array([])
    labels = np.argmax(o,axis=1)

    return labels

In [78]:
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

#  Train Neural Network

# set the number of nodes in input unit (not including bias unit)
n_input = train_data.shape[1]

# set the number of nodes in hidden unit (not including bias unit)
n_hidden = 50

# set the number of nodes in output unit
n_class = 10

# initialize the weights into some random matrices
initial_w1 = initializeWeights(n_input, n_hidden)
initial_w2 = initializeWeights(n_hidden, n_class)

# unroll 2 weight matrices into single column vector
initialWeights = np.concatenate((initial_w1.flatten(), initial_w2.flatten()), 0)

# set the regularization hyper-parameter
lambdaval = 0

args = (n_input, n_hidden, n_class, train_data, train_label, lambdaval)

# Train Neural Network using fmin_cg or minimize from scipy,optimize module. Check documentation for a working example

opts = {'maxiter': 50}  # Preferred value.

# Start time
start_time = time.time()
nn_params = minimize(nnObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)
# Ending time.
end_time = time.time()

# Difference between start and end-times.
time_dif = end_time - start_time

# Print the time-usage.
print("\nTime taken: " + str(timedelta(seconds=int(round(time_dif)))))

# Reshape nnParams from 1D vector into w1 and w2 matrices
w1 = nn_params.x[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
w2 = nn_params.x[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))

# Test the computed parameters

predicted_label = nnPredict(w1, w2, train_data)
accuracy = round(np.mean((predicted_label == train_label).astype(float))*100,2)
print('\nTraining set Accuracy:' + str(accuracy) + '%')


predicted_label = nnPredict(w1, w2, validation_data)
accuracy = round(np.mean((predicted_label == validation_label).astype(float))*100,2)
print('\nValidation set Accuracy:' + str(accuracy) + '%')


predicted_label = nnPredict(w1, w2, test_data)
accuracy = round(np.mean((predicted_label == test_label).astype(float))*100,2)
print('\nTest set Accuracy:' + str(accuracy) + '%')


preprocess done

Time taken: 0:00:28

Training set Accuracy:95.59%

Validation set Accuracy:94.82%

Test set Accuracy:95.29%
